# SPR 2026 - Ensemble Ponderado (Voting)

**Experimento #6 - Média Prioridade**

Média ponderada das probabilidades dos melhores modelos.

---
**CONFIGURAÇÃO:**
Este notebook assume que você já tem arquivos de predição (probabilidades) salvos.

**Pré-requisitos:**
1. Executar modelos individuais e salvar probabilidades como CSV
2. Combinar usando pesos baseados no F1-Score de validação
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
PROBA_DIR = '/kaggle/working'  # Ou onde estão salvas as probabilidades

test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f'Test: {test_df.shape}')

In [ ]:
# =================================================================
# OPÇÃO 1: Carregar probabilidades de arquivos salvos
# =================================================================
# Descomente e ajuste os paths conforme necessário

# proba_model1 = np.load(f'{PROBA_DIR}/proba_bertimbau.npy')
# proba_model2 = np.load(f'{PROBA_DIR}/proba_deberta.npy')
# proba_model3 = np.load(f'{PROBA_DIR}/proba_xlmroberta.npy')

# # Pesos baseados no F1-Score de validação
# weights = {
#     'bertimbau': 0.82,   # F1-Score val
#     'deberta': 0.81,
#     'xlmroberta': 0.80,
# }

# # Normalizar pesos
# total_weight = sum(weights.values())
# weights = {k: v / total_weight for k, v in weights.items()}

# # Média ponderada
# ensemble_proba = (
#     proba_model1 * weights['bertimbau'] +
#     proba_model2 * weights['deberta'] +
#     proba_model3 * weights['xlmroberta']
# )

# predictions = ensemble_proba.argmax(axis=1)

In [ ]:
# =================================================================
# OPÇÃO 2: Voting simples de predições (Hard Voting)
# =================================================================
# Combine predições de múltiplos arquivos de submissão

# sub1 = pd.read_csv(f'{PROBA_DIR}/submission_bertimbau.csv')
# sub2 = pd.read_csv(f'{PROBA_DIR}/submission_deberta.csv')
# sub3 = pd.read_csv(f'{PROBA_DIR}/submission_xlmroberta.csv')

# from scipy import stats

# all_preds = np.column_stack([
#     sub1['target'].values,
#     sub2['target'].values,
#     sub3['target'].values
# ])

# # Modo (voto majoritário)
# predictions = stats.mode(all_preds, axis=1)[0].flatten()

In [ ]:
# =================================================================
# EXEMPLO: Ensemble inline de TF-IDF + Modelo simples
# =================================================================
# Para demonstração, vamos criar um ensemble simples

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV

# Carregar dados
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

# TF-IDF
tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train = tfidf.fit_transform(train_df['report'])
X_test = tfidf.transform(test_df['report'])
y_train = train_df['target'].values

print(f'TF-IDF shape: {X_train.shape}')

In [ ]:
# Criar modelos
lr = LogisticRegression(C=1.0, max_iter=1000, class_weight='balanced', random_state=42)
svc = CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=1000, class_weight='balanced', random_state=42))

# Voting Classifier (Soft Voting)
ensemble = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('svc', svc),
    ],
    voting='soft',
    weights=[0.6, 0.4]  # Ajustar baseado no F1-Score de cada modelo
)

# Treinar
ensemble.fit(X_train, y_train)
print('Ensemble treinado!')

In [ ]:
# Predições
predictions = ensemble.predict(X_test)

# Submissão
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())